## Initialization code
Run these three cells after you plug the ADALM into your computer

In [ ]:
#import libraries needed for the code
import pysmu
from pysmu import Session, Mode, Device
import time
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os

In [ ]:
# initialize settings on the ADALM1000
session = Session(queue_size = 100000)
sample_rate = 100000
session.configure(sample_rate = sample_rate)
output = session.devices[0].channels['A'] #selects channel A as the one applying the stimulation
dev = session.devices[0]

In [1]:
class Channel:
    def __init__(self, channel_id, frequency=0.0, pulse_width=0.0, voltage=0.0, current=0.0, pulse_train_length=0, 
                 is_current_stim=False, num_of_stims=0):
        """
        Initialize a Channel object.

        Parameters:
        - channel_id (int): Unique identifier for the channel (1-16).
        - frequency (float): Frequency of the channel in Hz.
        - pulse_width (float): Pulse width of the channel in milliseconds.
        - voltage (float): Voltage of the channel in volts.
        - current (float): Current of the channel in amperes.
        - pulse_train_length (float): Duration of pulse train.
        - is_current_stim (bool): Toggle for defining the channel as current stimulation or voltage stimulation.
        - num_of_stims (int): Total number of pulses applied over 24hrs
        """
        if not 1 <= channel_id <= 16:
            raise ValueError("channel_id must be between 1 and 16")

        self.channel_id = channel_id
        self.frequency = frequency
        self.pulse_width = pulse_width
        self.voltage = voltage
        self.current = current
        self.pulse_train_length = pulse_train_length
        self.is_current_stim = is_current_stim
        self.num_of_stims = num_of_stims

    @property
    def duty_cycle(self):
        """Calculate and return the duty cycle as frequency * pulse_width / 1000."""
        return self.frequency * self.pulse_width / 1000

    def set_frequency(self, frequency):
        """Set the frequency of the channel."""
        if frequency < 0:
            raise ValueError("Frequency cannot be negative")
        self.frequency = frequency

    def set_pulse_width(self, pulse_width):
        """Set the pulse width of the channel."""
        if pulse_width < 0:
            raise ValueError("Pulse width cannot be negative")
        self.pulse_width = pulse_width

    def set_voltage(self, voltage):
        """Set the voltage of the channel."""
        if voltage < 0:
            raise ValueError("Voltage cannot be negative")
        self.voltage = voltage

    def set_current(self, current):
        """Set the current of the channel."""
        if current < 0:
            raise ValueError("Current cannot be negative")
        self.current = current

    def set_pulse_train_length(self, pulse_train_length):
        """Set the pulse train length of the channel."""
        if pulse_train_length < 0:
            raise ValueError("Pulse train length cannot be negative")
        self.pulse_train_length = pulse_train_length

    def set_current_stimulation_mode(self, is_current_stim):
        """Toggle the stimulation mode between current and voltage."""
        self.is_current_stim = is_current_stim

    def set_num_of_stims(self, num_of_stims):
        """Set the number of stims of the channel."""
        if num_of_stims < 0:
            raise ValueError("Number of stims cannot be negative")
        self.num_of_stims = num_of_stims
        
    def get_channel_info(self):
        """Return a dictionary containing the channel's attributes."""
        return {
            "channel_id": self.channel_id,
            "frequency": self.frequency,
            "pulse_width": self.pulse_width,
            "voltage": self.voltage,
            "current": self.current,
            "pulse_train_length": self.pulse_train_length,
            "is_current_stim": self.is_current_stim,
            "duty_cycle": self.duty_cycle,
        }

    def __repr__(self):
        stim_mode = "Current Stimulation" if self.is_current_stim else "Voltage Stimulation"
        return (
            f"Channel({self.channel_id}): Frequency={self.frequency} Hz, "
            f"Pulse Width={self.pulse_width} ms, Voltage={self.voltage} V, "
            f"Current={self.current} A, Pulse Train Length={self.pulse_train_length}, "
            f"Duty Cycle={self.duty_cycle:.2f}, Mode={stim_mode}, Number of Stims ={self.num_of_stims}"
        )


## Assign parameters for each channel
Each section below is a different channel, add channels and edit parameters as needed
Code will print out the parameters for each channel

In [3]:
number_of_channels = 1

# create a list of all channels 
channels = [Channel(channel_id=i) for i in range(1, number_of_channels+1)]

### Set attributes for a specific channel, change set_channel to the channel you want to set parameters for ###
### COPY CODE BELOW AND RENUMBER FOR EACH CHANNEL ###

set_channel = 1 # renumber here for each channel
channels[set_channel-1].set_frequency(10)
channels[set_channel-1].set_pulse_width(2)
channels[set_channel-1].set_voltage(2)
channels[set_channel-1].set_current(0)
channels[set_channel-1].set_pulse_train_length(15)
channels[set_channel-1].set_current_stimulation_mode(False)
channels[set_channel-1].set_num_of_stims(3)

### DONT EDIT BEYOND THIS POINT ###
#make boolean stim array for each time point
stim_array = []
times = []

for c, channel_name in enumerate(channels):
    times.append(channel_name.num_of_stims)
periods_off = []
for t, tim in enumerate(times):
    periods_off.append(max(times)/tim)
periods_off
for number in range(max(times)):
    on = []
    for value in periods_off:
        if (number)%value==0:
            on.append(True)
        else:
            on.append(False)
    stim_array.append(on)

# Print channel info
for channel_name in channels:
    print(channel_name)

Channel(1): Frequency=10 Hz, Pulse Width=2 ms, Voltage=2 V, Current=0 A, Pulse Train Length=15, Duty Cycle=0.02, Mode=Voltage Stimulation, Number of Stims =3


## Test the connection of each device
Running the code below will apply 100mV for 1ms to each device and record the average current see if your devices are shorted or disconnected 

In [ ]:
#assign arrays to each pin output
pin1 = [0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
pin2 = [0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1]
pin3 = [0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1]
pin4 = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]

#Test for shorting
n = 100
for c, channel_name in enumerate(channels):
    # set output of the mux to the channel that should be stimmed
    D0 = 0x50 + pin1[c] # 0x50 = set to 0, 0x51 = set to 1
    D1 = 0x50 + pin2[c]
    D2 = 0x50 + pin3[c]
    D3 = 0x50 + pin4[c]
    dev.ctrl_transfer(0x40, D0, 4, 0, 0, 0, 100) # set PIO 0
    dev.ctrl_transfer(0x40, D1, 5, 0, 0, 0, 100) # set PIO 1
    dev.ctrl_transfer(0x40, D2, 6, 0, 0, 0, 100) # set PIO 2
    dev.ctrl_transfer(0x40, D3, 7, 0, 0, 0, 100) # set PIO 3 
    
    output.mode = Mode.SVMI
    output.constant(0.1)
    recording = output.get_samples(n)
    # print(recording)
    currents = []
    for number in range(n):
        currents.append(recording[number][1])
    avg_current = sum(currents)/len(currents)
    print(avg_current)
    if avg_current > 0.01:
        print(f"Channel {c+1} is shorted")
    elif avg_current < 0.0004:
        print(f"Channel {c+1} is disconnected")
    else:
        print(f"Channel {c+1} is connected")
    output.constant(0)
    output.get_samples(10)

## Stimulate the Devices
This is the actual stimulation code, don't edit and don't run until you're sure everything is set up!

In [ ]:
#make new folder for stim data in the same location as this code
day = datetime.now().date()
newpath = f'{day}' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

#start the stimulation
for s in range(max(times)):
    stim_bool = stim_array[s]
    time_stimulating = 0
    #cycle through stimulations
    for c, channel_name in enumerate(channels):
        #determine if the channel should be stimulated at this time
        if stim_bool[c]:
            time_stimulating += channel_name.pulse_train_length
            # set output of the mux to the channel that should be stimmed
            D0 = 0x50 + pin1[c] # 0x50 = set to 0, 0x51 = set to 1
            D1 = 0x50 + pin2[c]
            D2 = 0x50 + pin3[c]
            D3 = 0x50 + pin4[c]
            dev.ctrl_transfer(0x40, D0, 4, 0, 0, 0, 100) # set PIO 0
            dev.ctrl_transfer(0x40, D1, 5, 0, 0, 0, 100) # set PIO 1
            dev.ctrl_transfer(0x40, D2, 6, 0, 0, 0, 100) # set PIO 2
            dev.ctrl_transfer(0x40, D3, 7, 0, 0, 0, 100) # set PIO 3 

            #apply either voltage or current stim 
            print(f"stim ch{c+1}")
            if channel_name.is_current_stim:
                output.mode = Mode.SIMV
                output.square(channel_name.current,0,sample_rate/channel_name.frequency,0,channel_name.duty_cycle)
                recording = output.get_samples(sample_rate*channel_name.pulse_train_length)
            else:
                output.mode = Mode.SVMI
                output.square(channel_name.voltage,0,sample_rate/channel_name.frequency,0,channel_name.duty_cycle)
                recording = output.get_samples(sample_rate*channel_name.pulse_train_length)

            #save the V and I traces from the stim to the folder created
            file = "Recording_Ch{}".format(c+1)
            now = datetime.now()
            now_formatted = now.strftime("%Y_%m_%d__%H_%M_%S")
            V = []
            A = []
            for i in range (sample_rate):
                V.append(recording[i][0])
                A.append(recording[i][1])
            df = pd.DataFrame({"V" : V, "A": A})
            df.to_csv(f"{day}/ch{c+1}_{now_formatted}.csv")

            #reset channel to 0V
            output.mode = Mode.SVMI
            output.constant(0)
            output.get_samples(1)
    #wait until next stimulation pulse            
    time.sleep((86400/max(times)-time_stimulating))

session.end()